## 캐싱(Caching)

LangChain은 LLM을 위한 선택적 캐싱 레이어를 제공합니다.

이는 두 가지 이유로 유용합니다.

- 동일한 완료를 여러 번 요청하는 경우 LLM 공급자에 대한 **API 호출 횟수를 줄여 비용을 절감**할 수 있습니다.
- LLM 제공업체에 대한 **API 호출 횟수를 줄여 애플리케이션의 속도를 높일 수** 있습니다.

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH04-Models")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH04-Models


모델과 프롬프트를 생성합니다


In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

# 모델을 생성합니다.
llm = ChatOpenAI(model_name="gpt-3.5-turbo")

# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template("{country} 에 대해서 200자 내외로 요약해줘")

# 체인을 생성합니다.
chain = prompt | llm

In [5]:
%%time 
response = chain.invoke({"country": "한국"})
print(response.content)

한국은 동아시아에 위치한 고도로 발전한 현대화된 국가이다. 수도는 서울로, 51.71 백만명이 거주하는 대도시이다. 한국은 사물인터넷 기술과 5G 네트워크 등 정보통신기술 분야에서 선두를 달리고 있다. 또한 한류와 K-pop은 전 세계적으로 큰 인기를 끌고 있으며 K-pop 아이돌은 아시아를 넘어서 글로벌한 영향력을 행사하고 있다. 또한 한국은 역사와 전통문화를 보존하고 있으며, 한복과 한국의 다양한 전통음식은 유명하다. 한반도에는 북한과의 군사적인 긴장 상태와 냉전 상태가 지속되고 있지만, 최근 평화적인 대화와 협력을 모색하고 있다.
CPU times: user 24.4 ms, sys: 6.14 ms, total: 30.5 ms
Wall time: 4.59 s


## InMemoryCache

인메모리 캐시를 사용하여 동일 질문에 대한 답변을 저장하고, 캐시에 저장된 답변을 반환합니다. 동일한 요청이 반복될 경우 캐시된 응답을 즉시 반환합니다.
프로그램 종료 시 메모리가 초기화되면서 캐시가 모두 사라지기 때문에 프로그램이 재시작되면 캐시를 다시 만들어야 합니다.

In [6]:
%%time
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache

# 인메모리 캐시를 사용합니다.
set_llm_cache(InMemoryCache())

# 체인을 실행합니다.
response = chain.invoke({"country": "한국"})
print(response.content)

한국은 동아시아에 위치한 고도로 발전한 경제를 갖고 있는 나라로, 국토는 작지만 인구 밀도가 높은 편이다. 수도는 서울이며, 고층건물과 현대적인 도시 풍경을 갖추고 있다. 역사적으로는 고조선 시대부터 이어져 온 민족의 역사가 있으며, 한글이라는 고유의 문자를 사용한다. 한국은 전통문화와 현대문화가 조화를 이루며 발전하고 있으며, K-pop, K-drama 등의 문화 콘텐츠로 세계적으로 인기를 얻고 있다. 또한 경제적으로도 세계 각국과의 무역이 활발하게 이루어지고 있으며, 한류 열풍을 통해 한국의 인지도가 전 세계적으로 높아지고 있다.
CPU times: user 119 ms, sys: 34.2 ms, total: 153 ms
Wall time: 5.42 s


In [8]:
%%time
# 체인을 실행합니다.
response = chain.invoke({"country": "한국"})
print(response.content)

한국은 동아시아에 위치한 고도로 발전한 경제를 갖고 있는 나라로, 국토는 작지만 인구 밀도가 높은 편이다. 수도는 서울이며, 고층건물과 현대적인 도시 풍경을 갖추고 있다. 역사적으로는 고조선 시대부터 이어져 온 민족의 역사가 있으며, 한글이라는 고유의 문자를 사용한다. 한국은 전통문화와 현대문화가 조화를 이루며 발전하고 있으며, K-pop, K-drama 등의 문화 콘텐츠로 세계적으로 인기를 얻고 있다. 또한 경제적으로도 세계 각국과의 무역이 활발하게 이루어지고 있으며, 한류 열풍을 통해 한국의 인지도가 전 세계적으로 높아지고 있다.
CPU times: user 2.72 ms, sys: 1.24 ms, total: 3.96 ms
Wall time: 3.42 ms


## SQLite Cache

데이터베이스 파일을 활용해 캐시 데이터를 저장하여 프로그램을 종료했다가 다시 시작해도 캐시 정보를 유지할 수 있습니다. 
동일한 질문에 대한 답변을 장기적으로 캐싱할 수 있어 재시작 후에도 캐싱된 답변을 빠르게 반환할 수 있습니다.

In [9]:
from langchain_community.cache import SQLiteCache
from langchain_core.globals import set_llm_cache
import os

# 캐시 디렉토리를 생성합니다.
if not os.path.exists("cache"):
    os.makedirs("cache")

# SQLiteCache를 사용합니다.
set_llm_cache(SQLiteCache(database_path="cache/llm_cache.db"))

In [14]:
%%time 
# 체인을 실행합니다.
response = chain.invoke({"country": "미국"})
print(response.content)

미국은 북아메리카 대륙의 주요 국가로, 면적과 인구 모두 세계에서 가장 큰 국가 중 하나이다. 50개의 주로 이루어져 있으며 미국엔 많은 다양한 인종과 문화가 살고 있다. 세계적으로 경제력과 군사력이 탁월하며, 주요한 국제기구 및 조직에서 영향력을 행사하고 있다. 미국의 수도는 워싱턴 D.C.이며, 뉴욕, 캘리포니아, 텍사스 등 주요 도시는 경제적으로 발전하고 다양한 문화를 지니고 있다. 미국은 자유로운 사회와 자본주의 원칙을 바탕으로 발전해 왔으며, 세계 각지에서 다양한 인종, 문화, 신념을 포용하는 다문화 사회로 알려져 있다.
CPU times: user 4.53 ms, sys: 2.89 ms, total: 7.42 ms
Wall time: 5.91 ms
